Thanks everyone for joining the google hangout on Feb 12, 2019 at 5 PM.  

This is a short demo to explain the code given in create_submission.py. Before that we'll answer couple of questions that were asked in the session.

Q1: Do we have to submit the code?

No, we only need the json containing the solution. create_submission.py script can help create the solution json file. While most of you will be trying programmatic approaches, it is perfectly fine to use crowdsourcing or BRAT tool to annotate the data too.

Q2: What should the output file contain?

It should be a json file containing a list of json elements, where each element corresponds to 1 sentence in the dataset.json input file. For each sentence, you have to provide a rewritten sentence (text), imputed entity (the one you used to replace XXXX), and the type(s) of the entity. Please use the create_submission.py to generate this.

In [5]:
import json
import random
import re

In [6]:
# this is the input file. it's a good idea to take a look.
dataset_filename = "./dataset.json"
with open(dataset_filename, 'r') as f:
    data = json.load(f)

In [7]:
print(len(data))

30916


In [8]:
# for analyzing, enough to look at only 10 rows
data = data[0:10]

In [9]:
print(len(data))

10


In [10]:
for text in data:
    print(text)

Let me explain to you why I believe this : For years, SunTrust advertised and promoted in big letters their XXXX XXXX debit card.
With that card, an account holder could earn XXXX mile for every {$1.00} spent ( for debit and credit card transactions ).
Over the past XXXX months, SunTrust must have reviewed their debit card portfolio and what they did as a result of their review is shocking : They closed all account holders who had earned above average number of miles regardless of the types of products they purchased, regardless of the length of their relationship with SunTrust, and regardless of the amount of money in their checking accounts.
After banking with Citibank for over 3 years, I discovered this evening that I could not use my debit card despite having a Citibank verified balance of {$900.00}. When I called Citibank, I was informed that the Texas branches had been bought by XXXX XXXX XXXX and as such, they, Citibank, could not allow access to my account.
I used to work for t

In [11]:
# lets take a look at the types.

pde_types_filename = "./types.json"
pde_types_list = []
with open(pde_types_filename, "r") as f:
    pde_types = json.load(f)
pde_types_list = pde_types['pde_types']

In [12]:
for pdet in pde_types_list:
    print(pdet)

/location
/location/address
/location/city
/location/zipcode
/finance/credit_score
/contact/name
/contact/email_address
/contact/phonenumber
/id/account_number
/id/us_social_security_number
/id/credit_card_number
/interest/product
/bio/age
/profession/job_title
/other/number
/other/datetime
/other/datetime/date
/other/datetime/month
/other/datetime/year
/other/organization
/other/organization/company


In [13]:
# the below is just copied from the create_submission.py


# this is not a complete list of patterns that you may find in original data (see ./additional_data)
# but we have chosen only these in the dataset for the hackathon
patterns = [r' XXXX XXXX XXXX XXXX', r'XXXX XXXX XXXX', r'XXXX XXXX', r'XXXX', r'XX/XX/XXXX', r'XX/XX/']
regex_patterns = '|'.join(patterns)

json_output = []
i = 1
for text in data:
    json_element = {}

    # we're using a simple regex pattern to find the redacted mentions
    mentions = re.findall(regex_patterns, text)
    if (len(mentions) != 1):
        print("Error! Only one entity mention per sentence should be present.")
        print(mentions)
        continue
    mention = mentions[0]

    ######### ENTITIES FOR DEMO - REPLACE THIS ##########
    entity = "entity_" + str(i)
    ######### ENTITIES FOR DEMO - REPLACE THIS ##########

    json_element['input_text'] = text
    json_element['text'] = text.replace(mention, entity)
    json_element["entity"] = entity

    ######### TYPES FOR DEMO - REPLACE THIS ##########
    # this are just some fake types that we are creating, you should remove all this
    rand_int = random.randint(0, len(pde_types_list)) - 1
    types = pde_types_list[rand_int]
    # typically parents of the types also apply to the entity, hence adding here
    if rand_int > 5:
       for pdet in pde_types:
          if pdet in types:
             types += " " + pdet
    # sometimes types from different branches in the hierarchy can apply to an entity
    if rand_int > 15:
        rand_int = random.randint(0, len(pde_types_list)) - 1
        another_type = pde_types_list[rand_int]
        if another_type not in types:
            types += " " + another_type
    ######### TYPES FOR DEMO - REPLACE THIS ##########

    json_element["types"] = types
    json_output.append(json_element)
i += 1

In [14]:
for json_element in json_output:
    print(json_element['input_text'])
    print(json_element['text'])
    print(json_element['entity'])
    print(json_element['types'])
    print("\n")

Let me explain to you why I believe this : For years, SunTrust advertised and promoted in big letters their XXXX XXXX debit card.
Let me explain to you why I believe this : For years, SunTrust advertised and promoted in big letters their entity_1 debit card.
entity_1
/finance/credit_score


With that card, an account holder could earn XXXX mile for every {$1.00} spent ( for debit and credit card transactions ).
With that card, an account holder could earn entity_1 mile for every {$1.00} spent ( for debit and credit card transactions ).
entity_1
/other/datetime/date /finance/credit_score


Over the past XXXX months, SunTrust must have reviewed their debit card portfolio and what they did as a result of their review is shocking : They closed all account holders who had earned above average number of miles regardless of the types of products they purchased, regardless of the length of their relationship with SunTrust, and regardless of the amount of money in their checking accounts.
Over 

In [15]:
# now let's see a quick and dirty way to annotate couple of sentences.

# you are expected to come with more rules like this, or use data programming (snorker)
# or use Machine Learning as per your convenience

for text in data:
    mentions = re.findall("XXXX mile|told XXXX", text)
    if(len(mentions) > 0):
        print(text)
        for mention in mentions:
            text = text.replace("XXXX mile", "9 mile")
            text = text.replace("told XXXX", "told Balaji")
            print(text)

With that card, an account holder could earn XXXX mile for every {$1.00} spent ( for debit and credit card transactions ).
With that card, an account holder could earn 9 mile for every {$1.00} spent ( for debit and credit card transactions ).
I told XXXX that I viewed this failure as additional negligence on the part of the bank.
I told Balaji that I viewed this failure as additional negligence on the part of the bank.
